# Abstractive Timeline Summarization

In [95]:
import pandas as pd
import numpy as np
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.metrics import pairwise_distances

from utils import process_parser_data

### Data parsing

In [96]:
dataset = process_parser_data('./datasets/meduza_19800101_20191002.csv')

b'Skipping line 57426: expected 17 fields, saw 19\n'


<class 'pandas.core.frame.DataFrame'>
Int64Index: 72569 entries, 49408 to 74060
Data columns (total 7 columns):
date       72569 non-null datetime64[ns]
url        72569 non-null object
edition    0 non-null object
title      72569 non-null object
text       72569 non-null object
authors    0 non-null float64
topics     0 non-null float64
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 4.4+ MB
None
                     date                                                url  \
49408 2014-09-12 09:04:00  https://meduza.io/news/2014/09/12/v-den-rozhde...   
49409 2014-10-07 13:19:00  https://meduza.io/news/2014/10/07/v-pravitelst...   
49410 2014-10-08 08:36:00  https://meduza.io/cards/7-russkiy-veb-chto-s-n...   
49411 2014-10-09 09:15:00  https://meduza.io/news/2014/10/09/sk-potrebova...   
49412 2014-10-10 08:27:00  https://meduza.io/cards/pridetsya-li-platit-za...   

      edition                                              title  \
49408    None  В день рождения Пут

### Feature generating

* TF-IDF for title and text
- Later will add embeddings

In [101]:
vectorizer = TfidfVectorizer(max_features=5000)
titles_tfidf = vectorizer.fit_transform(dataset['title'])
titles_tfidf

<72569x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 434215 stored elements in Compressed Sparse Row format>

### Calculating pairwise distances for clustering

In [103]:
titles_tfidf_sample = pd.DataFrame(titles_tfidf.toarray())[:train_size]

distances = pairwise_distances(titles_tfidf_sample,
                               metric='cosine',
                               n_jobs=3)
distances

MemoryError: Unable to allocate array with shape (20000, 20000) and data type float64

### Running clustering algorithm

In [ ]:
clustering = AgglomerativeClustering(
            affinity="precomputed",
            distance_threshold=0.04,
            n_clusters=None,
            linkage="average")

labels = clustering.fit_predict(distances)

KeyboardInterrupt: 

In [89]:
len(labels), len(set(labels))

(20000, 16110)

In [90]:
labels[labels == 0]

array([0, 0], dtype=int64)

In [94]:
print(*dataset[:train_size][labels == 3]['url'])

https://meduza.io/news/2014/10/22/kinokompaniyu-amedia-vystavili-na-prodazhu https://meduza.io/news/2015/12/10/restoran-praga-vystavili-na-prodazhu https://meduza.io/news/2016/01/10/osobnyak-osnovatelya-playboy-vystavyat-na-prodazhu https://meduza.io/news/2016/02/01/na-turetskih-kurortah-vystavili-na-prodazhu-1300-oteley
